# Homework 5 _ NLP
# Santiago Ruiz Piedrahita

HOMEWORK

I. Take the airline opinion dataset (airline_tweets.csv) that was the subject of the Class 12

II. Divide the dataset into Train/Test: 80/20 percent fix the randomness!

III. Compare the different Methods

IV. Compare the results in the table

V. Write conclusions

In [1]:
# librerias
import pandas as pd
import re
import seaborn as sns

# Limpiando los datos

In [2]:
data_source_url = "https://raw.githubusercontent.com/mhemmg/datasets/master/nlp/airline_tweets.csv"
airline_tweets = pd.read_csv(data_source_url)
airline_tweets2 = airline_tweets[['airline_sentiment','text']]
airline_tweets2.to_csv('full.csv',index=False)
airline_tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
features = airline_tweets2['text'].values
labels = airline_tweets2['airline_sentiment'].values

In [4]:
len(features)

14640

In [5]:
features

array(['@VirginAmerica What @dhepburn said.',
       "@VirginAmerica plus you've added commercials to the experience... tacky.",
       "@VirginAmerica I didn't today... Must mean I need to take another trip!",
       ...,
       '@AmericanAir Please bring American Airlines to #BlackBerry10',
       "@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??",
       '@AmericanAir we have 8 ppl so we need 2 know how many seats are on the next flight. Plz put us on standby for 4 people on the next flight?'],
      dtype=object)

In [6]:
labels

array(['neutral', 'positive', 'neutral', ..., 'neutral', 'negative',
       'neutral'], dtype=object)

In [7]:
processed_features = []

for i in range(0, len(features)):
    
    # remowing tweet
    processed_feature = re.sub(r'@\w+', ' ', str(features[i]))

    # remowing retweet
    processed_feature = re.sub(r'rt @\w+:', ' ', processed_feature)

    # Eliminación de números
    processed_feature = re.sub("\d+", ' ', processed_feature)

    # Removing links
    processed_feature = re.sub(r'http\S+', ' ', processed_feature)

    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', processed_feature)

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
#    processed_feature = re.sub(r'^b\s+', '', processed_feature)
    
    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)

In [8]:
airline_tweets['processed_features']=processed_features
airline_tweets2=airline_tweets[['airline_sentiment','text','processed_features']]
airline_tweets2.to_csv('full_clean.csv',index=False)

In [9]:
airline_tweets2

,airline_sentiment,text,processed_features
0,neutral,@VirginAmerica What @dhepburn said.,what said
1,positive,@VirginAmerica plus you've added commercials t...,plus you ve added commercials to the experien...
2,neutral,@VirginAmerica I didn't today... Must mean I n...,didn today must mean need to take another trip
3,negative,@VirginAmerica it's really aggressive to blast...,it really aggressive to blast obnoxious enter...
4,negative,@VirginAmerica and it's a really big bad thing...,and it a really big bad thing about it
...,...,...,...
14635,positive,@AmericanAir thank you we got on a different f...,thank you we got on different flight to chicago
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...,leaving over minutes late flight no warnings ...
14637,neutral,@AmericanAir Please bring American Airlines to...,please bring american airlines to blackberry
14638,negative,"@AmericanAir you have my money, you change my ...",you have my money you change my flight and do...


# Analisis de NLP

* ###  Linear vector classifier
* ###  Random forest
* ###  TextBlob - Naive Bayes


In [10]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

from nltk.corpus import stopwords

import spacy

from tqdm import tqdm

from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier
from textblob.classifiers import DecisionTreeClassifier

import nltk

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
nlp = spacy.load("en_core_web_sm")
spacy_stopwords_en = spacy.lang.en.stop_words.STOP_WORDS

In [14]:
vectorizer = TfidfVectorizer ( stop_words=spacy_stopwords_en)
processed_features_vec = vectorizer.fit_transform(processed_features).toarray()

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  % sorted(inconsistent)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(processed_features_vec, labels, test_size=0.2, random_state=0)

In [16]:
# Linear vector classifier
LSVC_classifier = LinearSVC(random_state=0, tol=1e-6)
LSVC_classifier.fit(X_train, y_train)

LSVC_predictions = LSVC_classifier.predict(X_test)
LSVC_acc = accuracy_score(y_test, LSVC_predictions)

print('Accuracy \t:',LSVC_acc)

Accuracy 	: 0.7708333333333334


In [17]:
# randon forest
RF_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
RF_classifier.fit(X_train, y_train)

RF_predictions = RF_classifier.predict(X_test)
RF_acc =accuracy_score(y_test, RF_predictions)

print('Accuracy \t:',RF_acc)


Accuracy 	: 0.7537568306010929


In [18]:
# TextBlob - Naive Bayes
X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

def new_format(X,Y):  
    return [(x,y) for x,y in zip(X,Y)]

def divide(data,n): 
    m = int(len(data)/n)
    return [data[i*m:(i+1)*m] for i in range(n)]

n_sets = 30
sets = divide(new_format(X_train,y_train),n_sets) 



In [19]:
cl = NaiveBayesClassifier(sets[0])
for i,j in zip(range(5),tqdm(range(5))): 
  cl.update(sets[i+1])

TB_acc = cl.accuracy(new_format(X_test,y_test))
print('Accuracy \t:',TB_acc)

 80%|████████  | 4/5 [00:58<00:14, 14.54s/it]


Accuracy 	: 0.76775956284153


## Testeando un ejemplo

In [20]:
dat = {"Metohd":["Linear vector classifier","Random Forest","TB naive Bayes"],"Accuracy":[LSVC_acc,RF_acc,TB_acc]}
pd.DataFrame(dat)

,Metohd,Accuracy
0,Linear vector classifier,0.770833
1,Random Forest,0.753757
2,TB naive Bayes,0.767760


In [23]:
# Analicemos las últimas filas 
test = airline_tweets2.tail()
test

,airline_sentiment,text,processed_features
14635,positive,@AmericanAir thank you we got on a different f...,thank you we got on different flight to chicago
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...,leaving over minutes late flight no warnings ...
14637,neutral,@AmericanAir Please bring American Airlines to...,please bring american airlines to blackberry
14638,negative,"@AmericanAir you have my money, you change my ...",you have my money you change my flight and do...
14639,neutral,@AmericanAir we have 8 ppl so we need 2 know h...,we have ppl so we need know how many seats ar...


In [34]:
features = test["processed_features"].values

features_v = vectorizer.transform(features)

In [35]:
testeoRF = []
testeoLSVC = []
testeoTB = []

for i in features_v:  
  testeoRF.append(RF_classifier.predict(i))
  testeoLSVC.append(LSVC_classifier.predict(i))


In [37]:
for i in features:   
  testeoTB.append(cl.classify(i))

In [39]:
test["Random Forest"]=testeoRF
test["Linear vector classifier"]=testeoLSVC
test["TB naive Bayes"]=testeoTB

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [40]:
test

,airline_sentiment,text,processed_features,Random Forest,Linear vector classifier,TB naive Bayes
14635,positive,@AmericanAir thank you we got on a different f...,thank you we got on different flight to chicago,[positive],[positive],neutral
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...,leaving over minutes late flight no warnings ...,[negative],[negative],negative
14637,neutral,@AmericanAir Please bring American Airlines to...,please bring american airlines to blackberry,[neutral],[neutral],neutral
14638,negative,"@AmericanAir you have my money, you change my ...",you have my money you change my flight and do...,[negative],[negative],negative
14639,neutral,@AmericanAir we have 8 ppl so we need 2 know h...,we have ppl so we need know how many seats ar...,[negative],[negative],negative


## Conclusion

Para responder las preguntas

* how woud you detect sentences showing: hatred/racism/sexism, in Twitter (for example)
* which method would you choose to clasify opinions on Amazon products: positive/negative

Con el reto de Bancolombia aprendí a hacer una clasificacion por medio de ZeroShot o de cosine similarity, esto añadido al analisis que se hace en este notebook, lo que se puede hacer por ejemplo es primero sacar los post negativos, seria una clasificacion para extraer lo negativo, y posteriormente se puede hacer otra clasificacion en dichos negativos por medio de los métodos vistos en la dadaton donde se usen un conjunto de datos que contenga las palabras que sean hatred/racism/sexism y asi detectarlas.